<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN03/3.2-Vorhersage_Hauspreise-PyTorch.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt


In [ ]:
# Imports
import torch
import pickle
import numpy as np
import pandas as pd
import torchmetrics
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, DataLoader, Subset

# Beispiel für eine Regression: Vorhersage der Kaufpreise von Häusern

Dieses Notebook enthält die Codebeispiele aus Kapitel 3, Abschnitt 5 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).

- Bei den beiden letzten Beispielen handelte es sich um **Klassifizierungsaufgaben**, die zum Ziel hatten, eine einzige Klassenbezeichnung eines Eingabedatenpunkts vorherzusagen. 
- Eine weitere häufige Machine-Learning-Aufgabe ist die **Regression**, bei der es darum geht, statt einer bestimmten Klassenbezeichnung stetige Werte vorherzusagen, z.B. die Prognose der morgigen Temperatur anhand von meteorologischen Daten oder eine Vorhersage des Zeitraums, der zum Abschluss eines Softwareprojekts erforderlich ist, anhand der Spezifikationen.

**Hinweis**
Verwechseln Sie die *Regression* nicht mit der *logistischen Regression*. Tatsächlich handelt es sich bei der logistischen Regression verwirrenderweise nicht um einen Regressionsalgorithmus, sondern um einen Klassifizierungsalgorithmus.

## Die Boston-Housing-Price-Datensammlung

- Wir werden versuchen, den durchschnittlichen Preis von in den Vorstädten von Boston gelegenen Immobilien anhand von Mitte der 1970er-Jahre erhobenen Daten über die Vorstädte vorherzusagen. 
- Dazu gehören beispielsweise die Kriminalitätsrate, die Höhe der Grundsteuer usw. 

Diese Datensammlung weist gegenüber den beiden vorangegangenen Beispielen einen interessanten Unterschied auf. Sie besteht aus relativ wenig Datenpunkten, nämlich aus nur 506, die in 404 Trainingsdatensätze und 102 Testdatensätze unterteilt sind. Die Merkmale der Eingabedaten (wie etwa die Kriminalitätsrate) sind von völlig verschiedenen Grössenordnungen.
Bei einigen dieser Merkmale handelt es sich um Verhältnisse oder Quoten, die Werte zwischen 0 und 1 annehmen können, andere liegen zwischen 1 und 12, wieder andere zwischen 0 und 100 usw.

In [ ]:
!wget -O boston_housing.pkl "https://github.com/ChristophWuersch/AppliedNeuralNetworks/raw/refs/heads/main/ANN02/Daten/boston_housing.pkl"

In [ ]:
# Re-import necessary libraries since execution state was reset
with open("boston_housing.pkl", "rb") as f:
    data = pickle.load(f)

train_data = data["train_data"]
train_targets = data["train_targets"]
test_data = data["test_data"]
test_targets = data["test_targets"]

train_data.shape

In [ ]:
test_data.shape

Wie Sie sehen, stehen 404 Trainingssamples und 102 Testsamples zur Verfügung,
die jeweils 13 numerische Merkmale besitzen, wie die Pro-Kopf-Kriminalitätsrate,
die durchschnittliche Anzahl der Zimmer pro Wohnung/Gebäude, ein Index, der
die Zugänglichkeit von Schnellstrassen erfasst, usw.


Die 13 Merkmale (features) in den Input-Daten sind wie folgt:

| feature no. | Beschreibung |
|:------------|:-------------|
| 1. | Per capita crime rate |
| 2. | Proportion of residential land zoned for lots over 25,000 square feet. |
| 3. | Proportion of non-retail business acres per town. |
| 4. | Charles River dummy variable (= 1 if tract bounds river; 0 otherwise). |
| 5. | Nitric oxides concentration (parts per 10 million). |
| 6. | Average number of rooms per dwelling. |
| 7. | Proportion of owner-occupied units built prior to 1940. |
| 8. | Weighted distances to five Boston employment centres. |
| 9. | Index of accessibility to radial highways. |
| 10. | Full-value property-tax rate per $10,000. |
| 11. | Pupil-teacher ratio by town. |
| 12. | 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town. |
| 13. | % lower status of the population.|

Die Zielvariablen sind die Medianwerte der Preise der von den Eigentümern selbst
genutzten Wohneinheiten in Einheiten von 1.000 Dollar:

In [ ]:
train_targets

- Die Preise liegen typischerweise zwischen 10.000 und 50.000 Dollar. Sie meinen, dass sei wenig? 
- Denken Sie daran, dass die Zahlen aus der Mitte der 1970er-Jahre stammen und nicht inflationsbereinigt sind.

## Daten vorbereiten

- Es würde zu Problemen führen, wenn man ein NN mit Werten füttert, die von völlig unterschiedlicher Grössenordnung sind. 
- as NN könnte sich möglicherweise automatisch an so ungleichartige Daten anpassen, aber das Lernen würde dadurch definitiv erschwert. 
- Zur Handhabung solcher Daten führt man üblicherweise eine **Normierung oder Standardisierung** (engl. Standardization, dt. auch als Normalisierung bezeichnet) der Merkmale durch: 
- Bei jedem Merkmal der Eingabedaten (eine Spalte in der Matrix der Eingabedaten) subtrahiert man den Mittelwert der gesamten Spalte und dividiert durch die Standardabweichung, sodass die Merkmalswerte um 0 zentriert sind und die Standardabweichung 1 besitzen. 

$$ \tilde{X}=\frac{X-\mu_X}{s_X}$$

In [ ]:
# Parameter des Transformer werden auf den Trainingsdaten bestimmt (gefittet, .fit in sklearn)
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# und diese Parameter werden verwendet, um die Testdaten zu skalieren.
test_data -= mean
test_data /= std

In [ ]:
# Convert to PyTorch tensors
train_data = torch.tensor(train_data, dtype=torch.float32)
train_targets = torch.tensor(train_targets, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)
test_targets = torch.tensor(test_targets, dtype=torch.float32)

print(train_data.shape, train_targets.shape)

**Beachten Sie hier, dass die zur Normierung der Testdaten verwendeten Werte mit
den Trainingsdaten berechnet werden.** Sie sollten in Ihrem Workflow niemals einen Wert verwenden, der anhand der Testdaten berechnet wird, auch nicht für etwas so Einfaches wie die Normierung der Daten.

## NN erzeugen

Weil nur wenige Samples verfügbar sind, erzeugen wir ein sehr kleines NN aus
zwei verdeckten Layern mit jeweils 64 Einheiten. 

- Im Allgemeinen gilt: Je weniger Trainingsdaten vorhanden sind, desto schlimmer wird sich die Überanpassung auswirken. Ein kleines NN ist eine Möglichkeit, die Überanpassung abzuschwächen.

Das gleiche Modell muss mehrmals instanziiert werden, daher wird zum
Erzeugen eine Funktion benutzt.

In [ ]:
class BostonHousingModel(pl.LightningModule):
    def __init__(self, input_dim):
        super(BostonHousingModel, self).__init__()

        # Define layers
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

        # Loss function
        self.loss_fn = nn.MSELoss()

        # Metric for Mean Absolute Error (MAE)
        self.mae = torchmetrics.MeanAbsoluteError()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_fn(y_pred, y.view(-1, 1))
        mae = self.mae(y_pred, y.view(-1, 1))
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_mae", mae, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_fn(y_pred, y.view(-1, 1))
        mae = self.mae(y_pred, y.view(-1, 1))
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_mae", mae, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.RMSprop(self.parameters(), lr=0.001)


### Mean Squared Error (`mse`) und Mean Absolute Error (`mae`)

- Beachten Sie hier, dass bei der Kompilierung der **mittlere quadratische Fehler
(`mse`, Mean Squared Error)**, also das Quadrat der Differenz zwischen Vorhersage und Zielvariable, als Verlustfunktion angegeben wird. 
- Diese Verlustfunktion ist bei Regressionsaufgaben sehr gebräuchlich.

$$\mathrm{mse} =\frac{1}{N} \sum_{i=1}^N \big\Vert y_i-\hat{y}_i \big\Vert^2_2  = \frac{1}{N}\sum_{i=1}^N \big\Vert y_i-\hat{y}(x_i)\big\Vert^2_2$$


- Das NN endet mit einer einzelnen Einheit und ohne Aktivierung (einem linearen Layer). 
- Dieser Aufbau ist typisch für skalare Regressionen (also Regressionen, die einen einzelnen stetigen Wert vorhersagen). Eine Aktivierungsfunktion würde den Bereich beschränken, den die Werte der Ausgabe annehmen können. 

Wenn Sie beispielsweise eine sigmoid-Aktivierungsfunktion auf den letzten Layer anwenden, könnte das Modell nur erlernen, Werte zwischen 0 und 1 vorherzusagen. Da der letzte Layer im vorliegenden Fall rein linear ist, kann das NN erlernen, Werte
aus einem beliebigen Bereich vorherzusagen.


- Ausserdem wird während des Trainings eine neue Kennzahl überwacht: der **mittlere absolute Fehler** (`mae`, Mean Absolute Error). 
- Hierbei handelt es sich um den Betrag der Differenz von Vorhersagen und Zielwerten. Ein Wert von 0.5 würde bei dieser Aufgabe bedeuten, dass die Vorhersagen durchschnittlich um 500 Dollar von den tatsächlichen Werten abweichen.

$$\mathrm{mae} =\frac{1}{N} \sum_{i=1}^N \vert y_i-\hat{y}_i \vert =\frac{1}{N} \sum_{i=1}^N \vert y_i-\hat{y}(x_i)\vert$$

In [ ]:
# Create PyTorch datasets
train_dataset = TensorDataset(train_data, train_targets)
test_dataset = TensorDataset(test_data, test_targets)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Instantiate model
input_dim = train_data.shape[1]
model = BostonHousingModel(input_dim=input_dim)

# Train the model using PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="auto",
    enable_progress_bar=False,
    logger=pl.loggers.CSVLogger("logs"),  # Save logs for plotting
)
trainer.fit(model, train_loader, val_loader)

In [ ]:
# Load logs from CSV for plotting
metrics = pd.read_csv("./logs/lightning_logs/version_0/metrics.csv")

metrics

In [ ]:
# Extract loss and MAE per epoch
train_loss = metrics["train_loss_epoch"].values
val_loss = metrics["val_loss"].values
train_mae = metrics["train_mae_epoch"].values
val_mae = metrics["val_mae"].values

In [ ]:
epochs = np.arange(1, len(train_loss) + 1)

# Plot Loss Curve
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label="Train Loss", marker="o")
plt.plot(epochs, val_loss, label="Validation Loss", marker="s")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plot MAE Curve
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_mae, label="Train MAE", marker="o", linestyle="dashed")
plt.plot(epochs, val_mae, label="Validation MAE", marker="s")
plt.xlabel("Epochs")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Training vs Validation MAE")
plt.legend()
plt.grid(True)
plt.show()


## K-fache Kreuzvalidierungen des Ansatzes

- Um die Leistung des NN zu beurteilen, während weiterhin eine Abstimmung der Parameter (wie etwa die Anzahl der Epochen beim Training) erfolgt, können Sie die Daten, wie beim letzten Beispiel, in eine Trainings- und eine Validierungsdatenmenge
aufteilen. 
- Da es jedoch so wenige Datenpunkte gibt, wäre die Validierungsmenge sehr klein (vielleicht nur rund 100 Samples). Dementsprechend könnten die Validierungsscores heftig schwanken, je nachdem, welche Datenpunkte Sie für die Validierung und für das Training auswählen. Die Validierungsscores würden bezüglich der Aufteilung eine hohe Varianz aufweisen, und das würde eine zuverlässige Beurteilung des Modells verhindern.

- Unter diesen Umständen hat sich die **K-fache Kreuzvalidierung** bewährt). Die verfügbaren Daten werden in $K$ Teilmengen aufgeteilt (typischerweise ist $K = 4$ oder $5$). Dann werden $K$ identische Modelle instanziiert, die mit jeweils $K – 1$ Teilmengen trainiert und anhand der verbleibenden Teilmenge beurteilt werden. 
- Der Validierungsscore des Modells ergibt sich dann als Durchschnittswert der $K$ erreichten Validierungsscores. 


<img src="Bilder/crossvalidation_3fold.png" width="840" align="center"/>

Als Code formuliert, ist das "straight forward".

In [ ]:
dataset = TensorDataset(train_data, train_targets)


# Define k-fold cross-validation
k = 4
num_epochs = 100
batch_size = 8
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_scores = []

for i, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"Processing fold #{i + 1}")

    # Create DataLoaders for training and validation sets
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = BostonHousingModel(input_dim=train_data.shape[1])

    # Train using PyTorch Lightning Trainer
    trainer = pl.Trainer(
        max_epochs=num_epochs,
        accelerator="auto",
        logger=False,
        enable_progress_bar=False,
    )
    trainer.fit(model, train_loader, val_loader)

    # Evaluate model on validation set
    val_loss = trainer.callback_metrics["val_loss"].item()
    val_mae = trainer.callback_metrics["val_mae"].item()
    all_scores.append(val_mae)

print("Cross-validation MAE scores:", all_scores)
print("Mean MAE:", sum(all_scores) / len(all_scores))


In [ ]:
all_scores

In [ ]:
np.mean(all_scores)

- Die verschiedenen Durchläufe liefern tatsächlich ziemlich unterschiedliche Validierungsscores, die von 2.6 bis 3.2 reichen. Der Durchschnittswert von 3.5 ist eine sehr viel verlässlichere Kennzahl als jeder einzelne Wert – und das ist ja letzten Endes auch der eigentliche Sinn der K-fachen Kreuzvalidierung. 

- In diesem Fall weichen die Vorhersagen durchschnittlich um 3.000 Dollar von den tatsächlichen Werten ab. In Anbetracht der Tatsache, dass die Preise zwischen 10.000 und 50.000 Dollar liegen, ist das kein unerheblicher Betrag.


Nun soll das NN etwas länger trainiert werden, nämlich 500 Epochen lang. Um aufzuzeichnen, wie gut das Modell in den verschiedenen Epochen funktioniert, können Sie die Trainingsschleife wie folgt modifizieren, sodass die Validierungsscores der Epochen protokolliert werden.

In [ ]:
# Create PyTorch dataset
dataset = TensorDataset(train_data, train_targets)

# Instantiate model
input_dim = train_data.shape[1]
model = BostonHousingModel(input_dim=input_dim)

In [ ]:
# K-Fold Cross Validation
k = 2
num_epochs = 500
batch_size = 8
kf = KFold(n_splits=k, shuffle=True, random_state=42)

train_mae_per_epoch = []
val_mae_per_epoch = []

fold_train_mae = []
fold_val_mae = []

for i, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"Processing fold #{i + 1}")

    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    model = BostonHousingModel(input_dim=train_data.shape[1])

    trainer = pl.Trainer(
        max_epochs=num_epochs,
        accelerator="auto",
        enable_progress_bar=False,
        logger=pl.loggers.CSVLogger("logs"),  # Save logs for plotting
    )

    trainer.fit(model, train_loader, val_loader)


In [ ]:
# Load logs from CSV for plotting
metrics = pd.read_csv("./logs/lightning_logs/version_1/metrics.csv") #TODO: 

metrics

In [ ]:
# Extract loss and MAE per epoch
train_loss = metrics["train_loss_epoch"].values
val_loss = metrics["val_loss"].values
train_mae = metrics["train_mae_epoch"].values
val_mae = metrics["val_mae"].values

In [ ]:
epochs = np.arange(1, len(train_loss) + 1)

# Plot Loss Curve
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label="Train Loss", marker="o")
plt.plot(epochs, val_loss, label="Validation Loss", marker="s")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

## Zusammenfassung

Nehmen Sie Folgendes aus diesem Abschnitt mit:
- Bei Regressionen kommen andere **Verlustfunktionen** als bei Klassifizierungen zum Einsatz. Der mittlere quadratische Fehler (Mean Squared Error, MSE) ist eine für Regressionen gebräuchliche Verlustfunktion.
- Auch die zur Beurteilung einer Regression gebräuchlichen Kennzahlen unterscheiden sich von denen, die für Klassifizierungen üblich sind. Das Konzept der Korrektklassifizierungsrate ist auf eine Regression nicht anwendbar. Stattdessen ist der mittlere absolute Fehler (Mean Absolute Error, MAE) eine gängige Kennzahl.
- Wenn die Merkmale der Eingabedaten von sehr unterschiedlicher Grössenordnung sind, sollten die Merkmale im Rahmen der Datenvorverarbeitung **voneinander unabhängig skaliert** (mittels der Trainingsdaten) werden.
- Falls nur wenige Daten vorliegen, können Sie zur verlässlicheren Beurteilung des Modells eine **$K$-fache Kreuzvalidierung** verwenden.
- Sollten nur wenig Trainingsdaten verfügbar sein, sollten kleine NN mit nur einigen wenigen verdeckten Layern (typischerweise ein oder zwei) verwendet werden, um eine **Überanpassung zu vermeiden**.

Sie sind nun in der Lage, die für Vektordaten gebräuchlichsten Machine-Learning-Aufgaben zu handhaben: 
1. **Binärklassifizierung**
2. **Mehrfachklassifizierung** und
3. **skalare Regression**. 




## Where's the intelligence?

- Für gewöhnlich ist es erforderlich, Rohdaten aufzubereiten (preprocessing, scaling, transforing,...), bevor man ein NN mit ihnen füttern kann.
- Wenn die Daten Merkmale von sehr unterschiedlicher Grössenordnung enthalten, sollten sie im Rahmen der Datenvorverarbeitung voneinander unabhängig skaliert werden.
- Bei fortgesetztem Training eines NNs kommt und mit zunehmender Modellkomplexität und abnehmender Datenmenge kommt es früher oder später zu einer **Überanpassung**, die zu schlechteren Ergebnissen bei noch unbekannten Daten führt.
- Falls nur wenige Trainingsdaten zur Verfügung stehen, sollten Sie ein kleines NN mit nur ein oder zwei verdeckten Layern verwenden, um eine Überanpassung zu vermeiden.
- Wenn Ihre Daten in sehr viele Kategorien unterteilt sind, verursachen Sie durch zu kleine zwischenliegende Layer womöglich Informationslecks.
- Bei Regressionen werden andere Verlustfunktionen und Leistungskennzahlen verwendet als bei Klassifizierungen.
- Bei kleinen Datenmengen kann die **$K$-fache Kreuzvalidierung** dabei helfen, ein Modell verlässlicher zu beurteilen. 
- Der Extremfall liegt vor bei $K=N$. In diesem Fall spricht man von der **Leave-One-Out-Kreuzvalidierung** (LOO=Leave One Out).